# Lab 4
The goals of this lab are as follows:
1. introduce criticality calculations; and
2. compare OpenMC criticality calculation for a simple geometry and material composition to 1-Group Critical Equation.

The problem is to find the critical radius of a solid sphere of pure Pu-239.  


## 1-Group Critical Equation
The analytic solution is arrived at by using the 1-Group Critical Equation given by:
$$B_1^2 = \frac{k_{\infty}-1}{L^2}$$
where $B_1^2$ is the buckling, and $L^2$ is the diffusion area; and $k_{\infty}$ is the infinite multiplication factor.  

The buckling depends on geometry; for this problem, the domain is spherical and buckling is given by:
$$B^{2}=\left( \frac{\pi}{\tilde{R}}\right)^{2}$$
where $\tilde{R}$ is the extrapolated radius of the sphere: $\tilde{R}=R+d$; with $d$ being the extrapolation distance.  The extrapolation distance is base on the diffusion coefficient: $d = 2.13D$ and the diffusion coefficient $(D)$ is based on the macroscopic transport cross section: $D=\frac{1}{3\Sigma_{tr}}$.

The macroscopic transport cross section is, in turn, computed from the corresponding microscopic cross section $(\sigma_{tr})$ and the atom density of the Pu-239 material.  

We're almost to the bottom of the rabbit hole...

The diffusion area is based on the diffusion coefficient and the macroscopic absorption cross section:
$$L^{2}=\frac{D}{\Sigma_{a}}$$
and, similar to the macroscopic transport cross section, $\Sigma_{a}$ is computed with the atom density of the Pu-239 material and the corresponding microscopic cross section $(\sigma_a)$

Lastly, $k_{\infty}$ is the infinite multiplication factor.  It is based on the regeneration factor ($\eta$) which we will take to be a known material property of Pu-239 and the fuel utilization, $f$, which we will take to be unity since the domain where neutrons will be diffusing and interacting is 100% fuel.

All of this long discussion is implemented in the code below.  Note that all elements of the equations above can be taken to be known *except* for the radius $R$.


In [1]:
import numpy as np

# data:
N_Pu = 0.037; #atoms/b-cm, atom density of Pu-239
sig_f = 1.85; #b, fission micro xc
sig_a = 2.11; #b, absorption micro xc
sig_tr = 6.8; #b, transport micro xc

Sig_tr = N_Pu*sig_tr; # 1/cm, transport macro xc
D = 1./(3.*Sig_tr); #cm, diffusion coefficient
d = 2.13*D; #cm, extrapolation length

f = 1.; # fuel utilization
eta_pu = 2.61; # 1-group reproduction factor for Pu-239

k_inf = f*eta_pu;

Sig_a = N_Pu*sig_a; # 1/cm, abs macro xc
L_sq = D/Sig_a; # cm**2, diffusion area
B_sq = (k_inf-1.)/L_sq; #1/cm**2, buckling

R_tild = np.pi/(np.sqrt(B_sq)); # cm, extrapolated radius

R = R_tild - d; # cm, 1-group critical radius

print(f'1-group critical radius for sphere of Pu-238: %5.4f cm'%R)






1-group critical radius for sphere of Pu-238: 7.3776 cm


## OpenMC model
Below we will implment this simple model and use OpenMC features to find the critical radius.

In [2]:
import openmc

In order to do this analysis we will use OpenMC's criticality search machinery.  The basic idea is that we need to create a function that takes one parameter - in this case $R$ - that will be varied while we search for the value to that parameter that results in $k_{eff}$ being equal to one.

In [3]:
# create the model.  "R" will be the parametric variable
def build_model(R):
    # create the sphere material
    fuel = openmc.Material(name='fuel');
    fuel.add_nuclide('Pu239',1);
    fuel.set_density('atom/b-cm',N_Pu);

    materials = openmc.Materials([fuel]); 
    
    fuel_sphere = openmc.Sphere(r=R,boundary_type='vacuum');
    
    fuel_cell = openmc.Cell();
    fuel_cell.fill = fuel;
    fuel_cell.region = -fuel_sphere;
    
    root_universe = openmc.Universe();
    root_universe.add_cells([fuel_cell]);
    
    geometry = openmc.Geometry(root_universe);
    
    settings = openmc.Settings()
    settings.batches = 200;
    settings.inactive = 50;
    settings.particles = 10000;
    
    bounds = [-R,-R,-R,R,R,R];
    uniform_dist = openmc.stats.Box(bounds[:3],bounds[3:],
                                    only_fissionable=True);
    settings.source = openmc.source.IndependentSource(space=uniform_dist);
    
    # so we don't waste disk input/output tallies that we will  not use
    settings.output = {'tallies':False};
    
    model = openmc.model.Model(geometry,materials,settings);
    
    return model

### Search for Critical Radius
To perform the search we will employ a call to 
<code> openmc.search_for_keff</code> function and pass in the relevant arguments.

In [4]:
crit_R, guesses, keffs = openmc.search_for_keff(build_model,
                                               bracket=[4.,15.],
                                               tol=1e-3,print_iterations=True)

print(f'Critical Radius: %5.4f'%crit_R);

[grayling:139617] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%

## Related Problem - Critical Dimension of a Cubic Thermal Reactor
For ER362 I often assign a homework problem in which the material composition of a thermal reactor is provided (usually just fuel and moderator; no structure, poison, etc...) and modified 1-group theory is used for the calculation.

Without repeating all of the details here, one problem is a bare cubic reactor in which U-235 and graphite are combined with a relative atom abundance of $1.0\times10^{-5}$ with the graphite, of course, being in the majority.  The 1-group theory result is that the side-length of the critical thermal reactor (at 20${^\circ}$C) is about 400 cm.  Let's see what OpenMC says with the help of the criticality search.

In [5]:
def thermal_model(L):
    fuel = openmc.Material(name='fuel');
    fuel.add_nuclide('U235',1.0e-5,'ao');
    fuel.add_nuclide('C0',1.,'ao');
    fuel.set_density('g/cm3',1.7); #assume the entire mixture has the density of pure graphite.
    fuel.add_s_alpha_beta('c_Graphite');
    fuel.temperature = 273.15 + 200.; # fuel at 200C expressed in K. 

    materials = openmc.Materials([fuel]); 
    
    # surfaces
    top = openmc.YPlane(y0=L/2.,boundary_type='vacuum');
    bottom = openmc.YPlane(y0=-L/2.,boundary_type='vacuum');
    front = openmc.XPlane(x0=L/2.,boundary_type='vacuum');
    back = openmc.XPlane(x0=-L/2.,boundary_type='vacuum');
    left = openmc.ZPlane(z0=-L/2.,boundary_type='vacuum');
    right = openmc.ZPlane(z0=L/2.,boundary_type='vacuum');
    
    core = openmc.Cell();
    core.fill = fuel
    core.region = -top & +bottom & -front & +back & +left & -right;
    
    root_universe = openmc.Universe();
    root_universe.add_cells([core]);
    
    geometry = openmc.Geometry(root_universe);
    
    settings = openmc.Settings()
    settings.batches = 200;
    settings.inactive = 50;
    settings.particles = 10000;
    settings.temperature['method']='interpolation'; # allow interpolation of temps.
    settings.temperature['multipole']=False; # specify use of windowed multipole data for resolved resonances.
    
    
    bounds = [-L/2.,-L/2.,-L/2.,L/2.,L/2.,L/2.];
    uniform_dist = openmc.stats.Box(bounds[:3],bounds[3:],
                                    only_fissionable=True);
    settings.source = openmc.source.IndependentSource(space=uniform_dist);
    
    # so we don't waste disk input/output tallies that we will  not use
    settings.output = {'tallies':False};
    
    model = openmc.model.Model(geometry,materials,settings);
    
    return model
    

In [6]:
crit_L, guesses, keffs = openmc.search_for_keff(thermal_model,
                                               bracket=[200.,500.],
                                               tol=1e-3,print_iterations=True)

print(f'Critical Side Length: %5.4f'%crit_L);

[grayling:140644] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%

This is not too far off from the 1-group result above.  Notice all of the gibberish below; I'm leaving this here just for posterity.  It turns out that the problem was that I'd used an unrealistically high mass density for the carbon (~2.6 g/cm3) vice the recommended density (from PNNL Material Handbook) of 1.7 g/cm3.  This *definitely* explains the problem and is a great example of what happens when you half-ass the QC on model inputs.  The good news is that the answer reinforces the correctness of the analytic result and, further, jibes with an intuitive expectation of how the OpenMC result *should* differ from the modified 1-group result.

__________________________________________________________________________________

The first time I successfully executed this critical search, I had left the fuel temperature at 20$^{\circ}$C when in the problem I am trying to emulate the fuel temperature is at 200$^{\circ}$C; a pretty important detail to leave off.  Nonetheless, the critical side length at 20$^{\circ}$C is about 278.4cm.  

Even getting the temperature correct, the critical side length is much smaller than modified 1-group theory predicts.  I'd like to know why.  Once answer might be: my modified 1-group theory analysis is just wrong.  Another answer might be that the neutron energy spectrum in the graphite reactor might deviate sufficiently from "thermal" spectrum that the "corrections" to diffusion parameters and reaction cross sections are incorrect.  Still, if I follow that logic - which would be to assume that neutrons are more fast than thermal - then I think that I should expect critical side length to be bigger rather than smaller.  Note that I added the setting specifying that windowed multipole expansions would be used for resolved resonances.  This didn't have much of an effect and, anyway, the only nuclide with resonance absorption is U235.  Using windowed multipole data, the critical side length is about 309.3 cm.  Not using it: 309.9. Conclusion: that's not it.

The problem is actually based on a question from Lamarsh (3rd ed.)- chapter 6, question 25; except for that question the specified temperature was 250$^{\circ}$C (I'm sure I changed it to 200 so there would be no need to interpolate tabulated values for non-1/v factors and regeneration factor.  I cannot use the "solution manual" for the text as backup because the "solution" for that problem is mostly missing from the PDF copy of the manual that I have. (*sigh*)